In [ ]:
from google.colab import drive
drive.mount("/gdrive", force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
4/4AEr08LffLUukSooE0PKb2_GH1Y8h1njQk5LCbwb19SApFXvS9roosA
Mounted at /gdrive


# **1. TF-IDF를 이용한 200개의 Keyword 추출**
**데이터 전처리**


In [ ]:
import numpy as np
import pandas as pd
import re

f = open('/content/drive/My Drive/colab/Clustering/SMS_dataset.txt','r',encoding='utf-8')
lines = f.readlines()

#1번 방법
corpus = []
for line in lines:
    contents = re.sub('[^a-z0-9]',' ',line.lower())  #소문자화, 알파벳/숫자 추출
    corpus.append(' '.join(contents.split()))  #이중 공백 제거

#2번 방법 (1/2번 중 하나만 사용하면 된다.)
regex = re.compile('[a-z0-9]+') #알파벳/숫자만 추출하는 정규식
doc2word = list()
for line in lines:            
    doc2word.append(regex.findall(line.lower()))  #소문자화
   
print(doc2word,'\n',len(doc2word))

[['go', 'until', 'jurong', 'point', 'crazy', 'available', 'only', 'in', 'bugis', 'n', 'great', 'world', 'la', 'e', 'buffet', 'cine', 'there', 'got', 'amore', 'wat'], ['ok', 'lar', 'joking', 'wif', 'u', 'oni'], ['free', 'entry', 'in', '2', 'a', 'wkly', 'comp', 'to', 'win', 'fa', 'cup', 'final', 'tkts', '21st', 'may', '2005', 'text', 'fa', 'to', '87121', 'to', 'receive', 'entry', 'question', 'std', 'txt', 'rate', 't', 'c', 's', 'apply', '08452810075over18', 's'], ['u', 'dun', 'say', 'so', 'early', 'hor', 'u', 'c', 'already', 'then', 'say'], ['nah', 'i', 'don', 't', 'think', 'he', 'goes', 'to', 'usf', 'he', 'lives', 'around', 'here', 'though'], ['freemsg', 'hey', 'there', 'darling', 'it', 's', 'been', '3', 'week', 's', 'now', 'and', 'no', 'word', 'back', 'i', 'd', 'like', 'some', 'fun', 'you', 'up', 'for', 'it', 'still', 'tb', 'ok', 'xxx', 'std', 'chgs', 'to', 'send', '1', '50', 'to', 'rcv'], ['even', 'my', 'brother', 'is', 'not', 'like', 'to', 'speak', 'with', 'me', 'they', 'treat', 'me'

**TF 계산**

In [ ]:
#[gensim corpora] *doc2word>> word corpus 들의 list
from gensim import corpora
word_dic = corpora.Dictionary(doc2word)
corpus = [word_dic.doc2bow(sentence) for sentence in doc2word]

tf_df = pd.DataFrame(columns=list(word_dic.token2id.keys()),index=['doc'+str(x) for x in range(len(doc2word))])

for doc_i in range(len(corpus)):
    for word in range(len(corpus[doc_i])):
        tf_df.iloc[doc_i][corpus[doc_i][word][0]] = corpus[doc_i][word][1]

tf_df

,amore,available,buffet,bugis,cine,crazy,e,go,got,great,in,jurong,la,n,only,point,there,until,wat,world,joking,lar,ok,oni,u,wif,08452810075over18,2,2005,21st,87121,a,apply,c,comp,cup,entry,fa,final,free,...,adventuring,chief,2667,3xx,gsex,wc1n,3mobile,bailiff,chatlines,inclu,l8er,servs,desk,mouse,childporn,belt,cribbs,hat,jumpers,spiritual,barring,influx,sudden,kane,shud,pshew,4years,accent,dental,nmde,units,dump,heap,lowes,salesman,087187272008,now1,pity,suggestions,bitching
doc0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
doc1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,1,1,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
doc2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,1,1,1,1,1,1,1,1,2,2,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
doc3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
doc4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
doc5567,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,NaN,NaN,NaN
doc5568,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
doc5569,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,NaN
doc5570,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [ ]:
# # corpora.Dictionary 사용법 
# print(word_dic.token2id,'\n\n')
# print(word_dic.token2id.values(),'\n\n')
# print(word_dic.token2id.keys(),'\n\n')

# print(type(word_dic.token2id))
# print(type(word_dic.token2id.values()))
# print(type(list(word_dic.token2id.keys())))


{'amore': 0, 'available': 1, 'buffet': 2, 'bugis': 3, 'cine': 4, 'crazy': 5, 'e': 6, 'go': 7, 'got': 8, 'great': 9, 'in': 10, 'jurong': 11, 'la': 12, 'n': 13, 'only': 14, 'point': 15, 'there': 16, 'until': 17, 'wat': 18, 'world': 19, 'joking': 20, 'lar': 21, 'ok': 22, 'oni': 23, 'u': 24, 'wif': 25, '08452810075over18': 26, '2': 27, '2005': 28, '21st': 29, '87121': 30, 'a': 31, 'apply': 32, 'c': 33, 'comp': 34, 'cup': 35, 'entry': 36, 'fa': 37, 'final': 38, 'free': 39, 'may': 40, 'question': 41, 'rate': 42, 'receive': 43, 's': 44, 'std': 45, 't': 46, 'text': 47, 'tkts': 48, 'to': 49, 'txt': 50, 'win': 51, 'wkly': 52, 'already': 53, 'dun': 54, 'early': 55, 'hor': 56, 'say': 57, 'so': 58, 'then': 59, 'around': 60, 'don': 61, 'goes': 62, 'he': 63, 'here': 64, 'i': 65, 'lives': 66, 'nah': 67, 'think': 68, 'though': 69, 'usf': 70, '1': 71, '3': 72, '50': 73, 'and': 74, 'back': 75, 'been': 76, 'chgs': 77, 'd': 78, 'darling': 79, 'for': 80, 'freemsg': 81, 'fun': 82, 'hey': 83, 'it': 84, 'like'

**IDF 계산**

In [ ]:
def calidf(term):
    import math
    docs = doc2word
    N = len(docs)
    df = 0

    for doc in docs:
        df += term in doc

    return math.log(N/df)

idf_list = [calidf(word) for word in list(word_dic.token2id.keys())]
idf_df = pd.DataFrame([idf_list for x in range(len(doc2word))],columns=list(word_dic.token2id.keys()),index=['doc'+str(x) for x in range(len(doc2word))])
idf_df

,amore,available,buffet,bugis,cine,crazy,e,go,got,great,in,jurong,la,n,only,point,there,until,wat,world,joking,lar,ok,oni,u,wif,08452810075over18,2,2005,21st,87121,a,apply,c,comp,cup,entry,fa,final,free,...,adventuring,chief,2667,3xx,gsex,wc1n,3mobile,bailiff,chatlines,inclu,l8er,servs,desk,mouse,childporn,belt,cribbs,hat,jumpers,spiritual,barring,influx,sudden,kane,shud,pshew,4years,accent,dental,nmde,units,dump,heap,lowes,salesman,087187272008,now1,pity,suggestions,bitching
doc0,8.625509,5.735138,7.932362,6.679599,6.679599,5.986452,4.11465,3.057165,3.196164,3.943378,1.934667,8.625509,6.679599,3.789227,3.278402,6.06056,3.264217,5.329672,4.000537,5.070161,6.83375,5.014591,2.994298,7.239215,1.908915,5.367413,7.932362,2.582877,7.526897,7.526897,7.239215,1.559896,5.159773,3.90701,6.140603,6.546068,5.580987,7.932362,5.735138,3.191787,...,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509
doc1,8.625509,5.735138,7.932362,6.679599,6.679599,5.986452,4.11465,3.057165,3.196164,3.943378,1.934667,8.625509,6.679599,3.789227,3.278402,6.06056,3.264217,5.329672,4.000537,5.070161,6.83375,5.014591,2.994298,7.239215,1.908915,5.367413,7.932362,2.582877,7.526897,7.526897,7.239215,1.559896,5.159773,3.90701,6.140603,6.546068,5.580987,7.932362,5.735138,3.191787,...,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509
doc2,8.625509,5.735138,7.932362,6.679599,6.679599,5.986452,4.11465,3.057165,3.196164,3.943378,1.934667,8.625509,6.679599,3.789227,3.278402,6.06056,3.264217,5.329672,4.000537,5.070161,6.83375,5.014591,2.994298,7.239215,1.908915,5.367413,7.932362,2.582877,7.526897,7.526897,7.239215,1.559896,5.159773,3.90701,6.140603,6.546068,5.580987,7.932362,5.735138,3.191787,...,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509
doc3,8.625509,5.735138,7.932362,6.679599,6.679599,5.986452,4.11465,3.057165,3.196164,3.943378,1.934667,8.625509,6.679599,3.789227,3.278402,6.06056,3.264217,5.329672,4.000537,5.070161,6.83375,5.014591,2.994298,7.239215,1.908915,5.367413,7.932362,2.582877,7.526897,7.526897,7.239215,1.559896,5.159773,3.90701,6.140603,6.546068,5.580987,7.932362,5.735138,3.191787,...,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509
doc4,8.625509,5.735138,7.932362,6.679599,6.679599,5.986452,4.11465,3.057165,3.196164,3.943378,1.934667,8.625509,6.679599,3.789227,3.278402,6.06056,3.264217,5.329672,4.000537,5.070161,6.83375,5.014591,2.994298,7.239215,1.908915,5.367413,7.932362,2.582877,7.526897,7.526897,7.239215,1.559896,5.159773,3.90701,6.140603,6.546068,5.580987,7.932362,5.735138,3.191787,...,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.625509,8.

**TFIDF 계산 및 각 단어의 Total Score 계산**

In [ ]:
tfidf_df = tf_df*idf_df
tfidf_df = tfidf_df.fillna(0)

total_score = []
for col in tfidf_df.columns:
    total_score.append(sum(tfidf_df[col]))

total_df = pd.DataFrame(total_score,columns=['TF-IDF'],index=list(word_dic.token2id.keys()))
total_df =total_df.sort_values('TF-IDF',ascending=False).head(200)
keyword = list(total_df.index)
total_df

# **K-Means Clustering**

**Document Vectorization**

In [110]:
doc2vec = tf_df[keyword].fillna(0)
X = [list(doc2vec.iloc[i]) for i in range(len(doc2word))]
doc2vec

,i,you,to,u,the,a,and,is,in,my,me,it,for,your,of,that,call,s,2,have,on,are,t,now,can,so,not,but,m,we,do,ur,or,at,if,be,will,get,with,no,...,ask,said,after,doing,b,yeah,life,really,im,why,them,meet,very,find,morning,babe,let,miss,would,last,thanks,cos,win,uk,nokia,also,anything,every,lol,sure,com,care,pick,150p,sent,urgent,keep,something,min,buy
doc0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
doc1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
doc2,0,0,3,0,0,1,0,0,1,0,0,0,0,0,0,0,0,2,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
doc3,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
doc4,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
doc5567,0,0,0,2,2,0,0,2,0,0,0,0,0,0,0,0,1,0,2,2,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
doc5568,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
doc5569,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
doc5570,2,0,1,0,1,0,1,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


**K-Means Clustering**

In [114]:
from sklearn.cluster import KMeans

km = KMeans(n_clusters=2)
y_km = km.fit_predict(X)

for i in range(len(doc2word)):
    print(str(i+1)+'번 문장:',str(y_km[i]+1)+'번 범주')

1번 문장: 2번 범주
2번 문장: 2번 범주
3번 문장: 2번 범주
4번 문장: 2번 범주
5번 문장: 2번 범주
6번 문장: 1번 범주
7번 문장: 2번 범주
8번 문장: 2번 범주
9번 문장: 2번 범주
10번 문장: 2번 범주
11번 문장: 1번 범주
12번 문장: 2번 범주
13번 문장: 2번 범주
14번 문장: 1번 범주
15번 문장: 2번 범주
16번 문장: 2번 범주
17번 문장: 2번 범주
18번 문장: 2번 범주
19번 문장: 2번 범주
20번 문장: 2번 범주
21번 문장: 2번 범주
22번 문장: 2번 범주
23번 문장: 2번 범주
24번 문장: 2번 범주
25번 문장: 2번 범주
26번 문장: 1번 범주
27번 문장: 2번 범주
28번 문장: 1번 범주
29번 문장: 1번 범주
30번 문장: 2번 범주
31번 문장: 2번 범주
32번 문장: 2번 범주
33번 문장: 2번 범주
34번 문장: 2번 범주
35번 문장: 2번 범주
36번 문장: 1번 범주
37번 문장: 2번 범주
38번 문장: 2번 범주
39번 문장: 2번 범주
40번 문장: 1번 범주
41번 문장: 2번 범주
42번 문장: 1번 범주
43번 문장: 2번 범주
44번 문장: 2번 범주
45번 문장: 1번 범주
46번 문장: 2번 범주
47번 문장: 2번 범주
48번 문장: 2번 범주
49번 문장: 2번 범주
50번 문장: 1번 범주
51번 문장: 2번 범주
52번 문장: 2번 범주
53번 문장: 2번 범주
54번 문장: 1번 범주
55번 문장: 2번 범주
56번 문장: 2번 범주
57번 문장: 2번 범주
58번 문장: 2번 범주
59번 문장: 2번 범주
60번 문장: 2번 범주
61번 문장: 2번 범주
62번 문장: 2번 범주
63번 문장: 2번 범주
64번 문장: 2번 범주
65번 문장: 2번 범주
66번 문장: 1번 범주
67번 문장: 2번 범주
68번 문장: 2번 범주
69번 문장: 2번 범주
70번 문장: 2번 범주
71번 문장: 2번 범주
72번 문장: 2번 범주
7